In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import socket, struct
import netaddr

import xxhash
%matplotlib inline

In [69]:
tp = pd.read_csv('capture20110818.pcap.netflow.labeled.csv', chunksize=1000)
df = pd.concat(tp, ignore_index=True)

# preprocessing
ips = df['SrcIPAddr:Port'].str.split(':')
df['SrcIPAddr'] = ips.str[0]
df['SrcIPPort'] = ips.str[-1]
ips = df['DstIPAddr:Port'].str.split(':')
df['DstIPAddr'] = ips.str[0]
df['DstIPPort'] = ips.str[-1]

df.head()

,Date,Flow_start,Durat,Prot,SrcIPAddr:Port,tmp,DstIPAddr:Port,Flags,Tos,Packets,Bytes,Flows,Label,Labels,SrcIPAddr,SrcIPPort,DstIPAddr,DstIPPort
0,2011-08-18,10:19:13.328,0.002,TCP,147.32.86.166:33426,->,212.24.150.110:25443,FRPA_,0,4,321,1,Background,NaN,147.32.86.166,33426,212.24.150.110,25443
1,2011-08-18,10:19:13.328,4.995,UDP,82.39.2.249:41915,->,147.32.84.59:43087,INT,0,617,40095,1,Background,NaN,82.39.2.249,41915,147.32.84.59,43087
2,2011-08-18,10:19:13.329,4.996,UDP,147.32.84.59:43087,->,82.39.2.249:41915,INT,0,1290,1909200,1,Background,NaN,147.32.84.59,43087,82.39.2.249,41915
3,2011-08-18,10:19:13.330,0.000,TCP,147.32.86.166:42020,->,147.32.192.34:993,A_,0,1,66,1,Background,NaN,147.32.86.166,42020,147.32.192.34,993
4,2011-08-18,10:19:13.330,0.000,TCP,212.24.150.110:25443,->,147.32.86.166:33426,FPA_,0,2,169,1,Background,NaN,212.24.150.110,25443,147.32.86.166,33426


In [70]:
confidence_interval = 0.99
error_rate = 0.001
w = int(np.ceil(2/error_rate)) #amount of 'counters' within a hash function
d = int(np.ceil(-np.log(1-confidence_interval)/np.log(2))) #amount of hash functions

print(w, d)


def update_wd(_hash,loc):
    wd[_hash][loc] = wd[_hash][loc] + 1
    return loc

2000 7


In [71]:
wd = np.zeros((d, w))
#create the count min table
for i in range(d):
    #compute hashes of each source ip
    df['hash' + str(i)] = df['SrcIPAddr'].apply(lambda row: xxhash.xxh32(row, seed=i).intdigest())
    #update row in count min table for these hashes 
    df['hash'+ str(i) + 'loc'] = df['hash' + str(i)].apply(lambda row: update_wd(i,row%w))
    
    
df.head()

,Date,Flow_start,Durat,Prot,SrcIPAddr:Port,tmp,DstIPAddr:Port,Flags,Tos,Packets,...,hash2,hash2loc,hash3,hash3loc,hash4,hash4loc,hash5,hash5loc,hash6,hash6loc
0,2011-08-18,10:19:13.328,0.002,TCP,147.32.86.166:33426,->,212.24.150.110:25443,FRPA_,0,4,...,1602117669,1669,1173467844,1844,3145439405,1405,4051052248,248,3152582910,910
1,2011-08-18,10:19:13.328,4.995,UDP,82.39.2.249:41915,->,147.32.84.59:43087,INT,0,617,...,523794326,326,4103352933,933,3627694176,176,1210628020,20,629576023,23
2,2011-08-18,10:19:13.329,4.996,UDP,147.32.84.59:43087,->,82.39.2.249:41915,INT,0,1290,...,164951896,1896,2222951432,1432,2722666753,753,2485698111,111,180318254,254
3,2011-08-18,10:19:13.330,0.000,TCP,147.32.86.166:42020,->,147.32.192.34:993,A_,0,1,...,1602117669,1669,1173467844,1844,3145439405,1405,4051052248,248,3152582910,910
4,2011-08-18,10:19:13.330,0.000,TCP,212.24.150.110:25443,->,147.32.86.166:33426,FPA_,0,2,...,3408876200,200,3059175285,1285,3535919388,1388,3910918517,517,2485411514,1514


In [87]:
print(wd)

[[3008. 2151.  206. ...  117.  173.  175.]
 [  79.   69.  996. ...  102.  276.   62.]
 [ 120.  263.   68. ...  158.   86.   96.]
 ...
 [ 135.   82.   66. ...   48. 2881.  127.]
 [ 142.  340.   95. ...  151. 7155.  250.]
 [  43.   65.  111. ...  293.  103.   83.]]
